# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab
import math
import random
import numpy
from matplotlib import pyplot as plt
%matplotlib inline

[INFO] 1451424787 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /Users/Alex/.graphlab/anaconda/lib/python2.7/site-packages/certifi/cacert.pem
1451424787 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to akorotcov@yahoo.com and will expire on September 23, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-2168 - Server binary: /Users/Alex/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1451424787.log
[INFO] GraphLab Server Version: 1.7.1


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [36]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 0.606306     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 0.655030     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [37]:
print model_all.get("coefficients").print_rows(num_rows = 18)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None |  274873.05595 |
|     bedrooms     |  None |      0.0      |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 8468.53108691 |
|   sqft_living    |  None | 24.4207209824 |
| sqft_living_sqrt |  None | 350.060553386 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 842.068034898 |
|    sqft_above    |  None | 20.0247224171 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [14]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [47]:
def l1_best(data, valid_data, test_data, features_list, output_name, penalties):
    # for each value of penalties, fit a model and compute average MSE
    l1_penalty_RSS = []
    total_nnz = []
    min_RSS = None
    best_l1_penalty = None
    for l1_penalty_tmp in penalties:
        model_tmp = graphlab.linear_regression.create(data, target=output_name, features=features_list,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty_tmp, verbose=False)
        
        predicted_tmp = model_tmp.predict(valid_data)
        RSS_tmp = ((predicted_tmp-valid_data[output_name])**2).sum()
        l1_penalty_RSS.append(RSS_tmp)
        total_nnz.append(model_tmp["coefficients"]['value'].nnz())
        if min_RSS is None or RSS_tmp < min_RSS:
            min_RSS = RSS_tmp
            best_l1_penalty = l1_penalty_tmp            
    best_model = graphlab.linear_regression.create(data, target=output_name, features=features_list,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=best_l1_penalty, verbose=False)
    predicted_test = best_model.predict(test_data)
    RSS_test = ((predicted_test-test_data[output_name])**2).sum()
    return (l1_penalty_RSS, best_l1_penalty, RSS_test, best_model, total_nnz)

In [48]:
# Create polynomial features
features_all = all_features # set the name of the features
target = 'price'# assign target name
#Create list of penalties
penalties = numpy.logspace(1, 7, num=13)
# Compute best model
best_fit = l1_best(training, validation, testing, features_all, target, penalties)
print "Penalties: ", penalties
print "Errors: ", best_fit[0]
print "Best penalty =", best_fit[1]

Penalties:  [  1.00000000e+01   3.16227766e+01   1.00000000e+02   3.16227766e+02
   1.00000000e+03   3.16227766e+03   1.00000000e+04   3.16227766e+04
   1.00000000e+05   3.16227766e+05   1.00000000e+06   3.16227766e+06
   1.00000000e+07]
Errors:  [625766285142459.9, 625766285362394.1, 625766286057885.0, 625766288257224.6, 625766295212186.8, 625766317206080.5, 625766386760658.1, 625766606749278.5, 625767302791634.1, 625769507643886.2, 625776517727024.0, 625799062845467.0, 625883719085425.2]
Best penalty = 10.0


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [49]:
print "Best l1_penalty is", best_fit[1]
print "Best RSS on test data is", best_fit[2]

Best l1_penalty is 10.0
Best RSS on test data is 1.56983602382e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [46]:
best_fit[3].get("coefficients").print_rows(num_rows = 18)
print "Total non-zero weights", best_fit[3]["coefficients"]['value'].nnz()

+------------------+-------+------------------+
|       name       | index |      value       |
+------------------+-------+------------------+
|   (intercept)    |  None |  18993.4272128   |
|     bedrooms     |  None |  7936.96767903   |
| bedrooms_square  |  None |  936.993368193   |
|    bathrooms     |  None |  25409.5889341   |
|   sqft_living    |  None |  39.1151363797   |
| sqft_living_sqrt |  None |  1124.65021281   |
|     sqft_lot     |  None | 0.00348361822299 |
|  sqft_lot_sqrt   |  None |  148.258391011   |
|      floors      |  None |   21204.335467   |
|  floors_square   |  None |  12915.5243361   |
|    waterfront    |  None |  601905.594545   |
|       view       |  None |  93312.8573119   |
|    condition     |  None |  6609.03571245   |
|      grade       |  None |  6206.93999188   |
|    sqft_above    |  None |  43.2870534193   |
|  sqft_basement   |  None |  122.367827534   |
|     yr_built     |  None |  9.43363539372   |
|   yr_renovated   |  None |  56.0720034

In [50]:
print "Total non-zero for each penalty", best_fit[4]

Total non-zero for each penalty [18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18]


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [38]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [66]:
l1_penalty_values = numpy.logspace(8, 10, num=20)
print l1_penalty_values

[  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]


Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [51]:
best_fit_new = l1_best(training, validation, testing, features_all, target, l1_penalty_values)

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [67]:
l1_penalty_min = None
l1_penalty_max = None
for ind_tmp in range(len(best_fit_new[0])):
    print "penalty", l1_penalty_values[ind_tmp], "non-zero", best_fit_new[4][ind_tmp]
    if best_fit_new[4][ind_tmp] >= max_nonzeros:
        l1_penalty_max = l1_penalty_values[ind_tmp]
    if best_fit_new[4][ind_tmp] <= max_nonzeros and l1_penalty_min < best_fit_new[4][ind_tmp]:
        l1_penalty_min = l1_penalty_values[ind_tmp]
print "l1_penalty_min", l1_penalty_min
print "l1_penalty_max", l1_penalty_max

penalty 100000000.0 non-zero 18
penalty 127427498.57 non-zero 18
penalty 162377673.919 non-zero 18
penalty 206913808.111 non-zero 18
penalty 263665089.873 non-zero 17
penalty 335981828.628 non-zero 17
penalty 428133239.872 non-zero 17
penalty 545559478.117 non-zero 17
penalty 695192796.178 non-zero 17
penalty 885866790.41 non-zero 16
penalty 1128837891.68 non-zero 15
penalty 1438449888.29 non-zero 15
penalty 1832980710.83 non-zero 13
penalty 2335721469.09 non-zero 12
penalty 2976351441.63 non-zero 10
penalty 3792690190.73 non-zero 6
penalty 4832930238.57 non-zero 5
penalty 6158482110.66 non-zero 3
penalty 7847599703.51 non-zero 1
penalty 10000000000.0 non-zero 1
l1_penalty_min 3792690190.73
l1_penalty_max 2976351441.63


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [68]:
l1_penalty_values_narrow = numpy.linspace(l1_penalty_min,l1_penalty_max,20)
print l1_penalty_values

[  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]


* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [69]:
best_fit_new_narrow = l1_best(training, validation, testing, features_all, target, l1_penalty_values_narrow)

In [75]:
final_penalties = []
for ind_tmp in range(len(best_fit_new_narrow[0])):
    print "penalty", l1_penalty_values_narrow[ind_tmp], "non-zero", best_fit_new_narrow[4][ind_tmp]
    if best_fit_new_narrow[4][ind_tmp] == max_nonzeros:
        final_penalties.append(l1_penalty_values_narrow[ind_tmp])
numpy.asarray(final_penalties)
print final_penalties

penalty 3792690190.73 non-zero 6
penalty 3749724993.41 non-zero 6
penalty 3706759796.09 non-zero 6
penalty 3663794598.77 non-zero 6
penalty 3620829401.45 non-zero 6
penalty 3577864204.13 non-zero 7
penalty 3534899006.81 non-zero 7
penalty 3491933809.48 non-zero 7
penalty 3448968612.16 non-zero 7
penalty 3406003414.84 non-zero 8
penalty 3363038217.52 non-zero 8
penalty 3320073020.2 non-zero 8
penalty 3277107822.88 non-zero 10
penalty 3234142625.56 non-zero 10
penalty 3191177428.24 non-zero 10
penalty 3148212230.92 non-zero 10
penalty 3105247033.59 non-zero 10
penalty 3062281836.27 non-zero 10
penalty 3019316638.95 non-zero 10
penalty 2976351441.63 non-zero 10
[3577864204.1267428, 3534899006.8056407, 3491933809.4845386, 3448968612.1634364]


In [76]:
best_fit_final = l1_best(training, validation, testing, features_all, target, final_penalties)

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [79]:
print "validation errors", best_fit_final[0]
print "value of l1_penalty in our narrow range has with the lowest RSS on the VALIDATION", best_fit_final[1]
best_fit_final[3].get("coefficients").print_rows(num_rows = 18)

validation errors [1060799531763287.8, 1055992735342999.1, 1051147625612860.9, 1046937488751711.1]
value of l1_penalty in our narrow range has with the lowest RSS on the VALIDATION 3448968612.16
+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None | 222253.192544 |
|     bedrooms     |  None | 661.722717782 |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 15873.9572593 |
|   sqft_living    |  None | 32.4102214513 |
| sqft_living_sqrt |  None | 690.114773313 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 2899.42026975 |
|    sqft_above    |  None | 30.01157530